# Setup

In [1]:
import numpy as np
import mnist
from tensorflow import keras

# The first time you run this might be a bit slow, since the
# mnist package has to download and cache the data.
train_images = mnist.train_images()
train_labels = mnist.train_labels()

print(train_images.shape) # (60000, 28, 28)
print(train_labels.shape) # (60000,)

2022-10-08 19:39:57.807434: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


(60000, 28, 28)
(60000,)


# Preparing the Data

In [8]:
train_images = mnist.train_images()
train_labels = mnist.train_labels()
test_images = mnist.test_images()
test_labels = mnist.test_labels()

# Normalize the images.
train_images = (train_images / 255) - 0.5
test_images = (test_images / 255) - 0.5

In [9]:
train_labels

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [10]:
train_images.shape

(60000, 28, 28)

In [11]:
# Reshape the images.
train_images = np.expand_dims(train_images, axis=3)
test_images = np.expand_dims(test_images, axis=3)

print(train_images.shape) # (60000, 28, 28, 1)
print(test_images.shape)  # (10000, 28, 28, 1)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


# Building Model

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

num_filters = 8
filter_size = 3
pool_size = 2

model = Sequential([
    Conv2D(num_filters, filter_size, input_shape=(28, 28, 1)),
    MaxPooling2D(pool_size=pool_size),
    Flatten(),
    Dense(10, activation='softmax')
])

model.compile(
    'adam',
    loss='categorical_crossentropy',
    metrics='accuracy'
)

# Fitting

In [19]:
from tensorflow.keras.utils import to_categorical

model.fit(
    train_images,
    to_categorical(train_labels),
    epochs=3,
    validation_data=(test_images, to_categorical(test_labels))
)

Epoch 1/3
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3445 - accuracy: 0.9018 - val_loss: 0.1875 - val_accuracy: 0.9471
Epoch 2/3
1875/1875 [==============================] - 8s 4ms/step - loss: 0.1635 - accuracy: 0.9533 - val_loss: 0.1322 - val_accuracy: 0.9602
Epoch 3/3
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1168 - accuracy: 0.9668 - val_loss: 0.1036 - val_accuracy: 0.9693


# Saving Model

In [23]:
model.save("bleh.h5")
reconstructed_model = keras.models.load_model("bleh.h5")

In [26]:
np.testing.assert_allclose(
    model.predict(test_images), reconstructed_model.predict(test_images)
)

313/313 [==============================] - 1s 2ms/step


In [33]:
np.argmax(model.predict(test_images[0:1,:,:,:]))

1/1 [==============================] - 0s 15ms/step


7